In [254]:
from bs4 import BeautifulSoup
import urllib.request
import time
import json

In [255]:
def get_crime_zipcode_mapping():
    list_urls = get_all_urls()
    ret_dict = {}
    for url in list_urls:
        wp = urllib.request.urlopen(url)
        soup = BeautifulSoup(wp, 'html.parser')
        tables = soup.findAll("table")
        tags = tables[1].findAll("td")
        property_crime = tags[-2].string
        violent_crime = tags[-5].string
        url_split = url.split("/")
        zipcode = url_split[-1]
        city_name = url_split[-2]
        if (property_crime == None):
            default_dict = {"property_crime": "43.5", "violent_crime": "41.4"}
            ret_dict[str((city_name, zipcode))] = default_dict
        else:
            new_dict = {"property_crime": property_crime, "violent_crime": violent_crime}
            ret_dict[str((city_name, zipcode))] = new_dict
    return ret_dict

In [256]:
def get_zipcodes_by_city(city_name):
    with open('./zipcodes_by_city.json') as data_file:    
        data = json.load(data_file)
    return data[city_name]

In [257]:
base_url = "http://www.bestplaces.net/crime/zip-code/illinois"
city_list = ['San Francisco', 'Austin', 'Chicago', 'New York']
city_list_to_url = {"San Francisco": 'san_francisco', "Austin": "austin", "Chicago": "chicago", "New York": "new_york"}

In [258]:
def get_all_urls():
    ret_urls = []
    for city in city_list:
        city_zipcodes = get_zipcodes_by_city(city)
        for zipcode in city_zipcodes:
            parameters = "/{city_code}/{zip_code}".format(city_code=city_list_to_url[city], zip_code=zipcode)
            url = base_url + parameters
            ret_urls.append(url)
    return ret_urls

In [259]:
with open('./zipcode_crime.json', 'w') as outfile:
    data = get_crime_zipcode_mapping()
    json.dump(data, outfile)